In [ ]:
import numpy as np
import sklearn.cluster
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.animation as animation
from matplotlib.cm import get_cmap
from matplotlib import colors
from matplotlib.colors import ListedColormap
%matplotlib inline
plt.rcParams["animation.html"] = "jshtml"

In [ ]:
def calculate_connectivity_matrix(u,v,hrus,uhrus):
    nhrus = len(uhrus)
    gamma_in_x = np.zeros((nhrus,nhrus)) #(j to i)
    gamma_in_y = np.zeros((nhrus,nhrus))
    gamma_out_x = np.zeros((nhrus,nhrus)) #(i to j)
    gamma_out_y = np.zeros((nhrus,nhrus))
    #W u>0
    hrus_w = hrus[:,1:]
    w_w = np.zeros((nhrus,nhrus))
    for i in range(nhrus):
        m = (hrus==uhrus[i])& (u[i]>0)
        m = m[:,:-1]
        if np.sum(m)>0:
            m_array = hrus_w[m]
            #print(m_array)
            for j in range(nhrus):
                if uhrus[j] in m_array:
                    if i!=j:
                        gamma_out_x[i,j] +=  (m_array==uhrus[j]).sum()
    #E u<0      
    hrus_e = hrus[:,:-1]
    w_e = np.zeros((nhrus,nhrus))
    for i in range(nhrus):
        m = (hrus==uhrus[i])& (u[i]<0)
        m = m[:,1:]
        if np.sum(m)>0:
            m_array = hrus_e[m]
            #print(m_array)
            for j in range(nhrus):
                if uhrus[j] in m_array:
                    if i!=j:
                        gamma_out_x[i,j] +=  (m_array==uhrus[j]).sum()

    #S v>0
    hrus_s = hrus[1:,:]
    w_s = np.zeros((nhrus,nhrus))
    for i in range(nhrus):
        m = (hrus==uhrus[i])& (v[i]>0)
        m = m[:-1,:]
        if np.sum(m)>0:
            m_array = hrus_s[m]
            #print(m_array)
            for j in range(nhrus):
                if uhrus[j] in m_array:
                    if i!=j:
                        gamma_out_y[i,j] +=  (m_array==uhrus[j]).sum()
    #N v<0      
    hrus_n = hrus[:-1,:]
    w_n = np.zeros((nhrus,nhrus))
    for i in range(nhrus):
        m = (hrus==uhrus[i])& (v[i]<0)
        m = m[1:,:]
        if np.sum(m)>0:
            m_array = hrus_n[m]
            #print(m_array)
            for j in range(nhrus):
                if uhrus[j] in m_array:
                    if i!=j:
                        gamma_out_y[i,j] +=  (m_array==uhrus[j]).sum()

    ## in
    #W u>0
    hrus_w = hrus[:,:-1]
    w_w = np.zeros((nhrus,nhrus))
    for i in range(nhrus):
        m = hrus==uhrus[i]
        m = m[:,1:]
        if np.sum(m)>0:
            m_array = hrus_w[m]
            #print(m_array)
            for j in range(nhrus):
                if u[j]>0:
                    if uhrus[j] in m_array:
                        if i!=j:
                            gamma_in_x[i,j] +=  (m_array==uhrus[j]).sum()
    #E u<0      
    hrus_e = hrus[:,1:]
    w_e = np.zeros((nhrus,nhrus))
    for i in range(nhrus):
        m = (hrus==uhrus[i])
        m = m[:,:-1]
        if np.sum(m)>0:
            m_array = hrus_e[m]
            #print(m_array)
            for j in range(nhrus):
                if u[j]<0:
                    if uhrus[j] in m_array:
                        if i!=j:
                            gamma_in_x[i,j] +=  (m_array==uhrus[j]).sum()

    #S v>0
    hrus_s = hrus[:-1,:]
    w_s = np.zeros((nhrus,nhrus))
    for i in range(nhrus):
        m = hrus==uhrus[i]
        m = m[1:,:]
        if np.sum(m)>0:
            m_array = hrus_s[m]
            #print(m_array)
            for j in range(nhrus):
                if v[j]>0:
                    if uhrus[j] in m_array:
                        if i!=j:
                            gamma_in_y[i,j] +=  (m_array==uhrus[j]).sum()
    #N v<0      
    hrus_n = hrus[1:,:]
    w_n = np.zeros((nhrus,nhrus))
    for i in range(nhrus):
        m = hrus==uhrus[i]
        m = m[:-1,:]
        if np.sum(m)>0:
            m_array = hrus_n[m]
            #print(m_array)
            for j in range(nhrus):
                if v[j]<0:
                    if uhrus[j] in m_array:
                        if i!=j:
                            gamma_in_y[i,j] +=  (m_array==uhrus[j]).sum()   
                        
    gamma = np.zeros((4,nhrus,nhrus))
    gamma[0] = gamma_in_x
    gamma[1] = gamma_out_x
    gamma[2] = gamma_in_y
    gamma[3] = gamma_out_y
    return gamma

## Example 1

In [ ]:
# Define simulation parameters
Xn = 6
dx = 30   
dy = 30  
T = 200 
dt = 1
nts= int(T/dt)


# generate HRU
k = 8  # Define the number of clusters
lat = np.arange(Xn)
lon = np.arange(Xn)
lats, lons = np.meshgrid(lon, lat)
valr = np.random.normal(0, Xn, (Xn,Xn))
X = np.concatenate((lats.reshape(1,Xn*Xn),lons.reshape(1,Xn*Xn),valr.reshape(1,Xn*Xn)), axis=0).T
np.random.seed(1)
model = sklearn.cluster.KMeans(n_clusters=k,n_init='auto')
#Fit the model
model.fit(X)
#Predict the clusters on all the data
clusters = model.predict(X)
hrus = clusters.reshape(Xn,Xn)

uhrus = np.unique(hrus)[:].astype(int)
zi_hrus = np.zeros(len(uhrus))+1
# initial concentration
c_hrus = np.zeros((nts,len(uhrus)))
c_hrus[0,6] = 1
# wind
u_hrus = np.zeros(len(uhrus))+1
v_hrus = np.zeros(len(uhrus))+1

hru_area = np.zeros(len(uhrus))
for i in range(len(uhrus)):
    hru = uhrus[i]
    hru_area[i] = hrus[hrus==hru].shape[0]



# HRU advection
gamma = calculate_connectivity_matrix(u_hrus,v_hrus,hrus,uhrus)
for n in range(nts-1):
    fin = np.zeros(len(uhrus))
    fout = np.zeros(len(uhrus))
    for a in range(len(uhrus)):
        fout[a] = np.abs(u_hrus[a])*c_hrus[n][a]*zi_hrus[a]*np.sum(gamma[1][a,:]*dx)/hru_area[a]/dx/dy/zi_hrus[a] + \
                  np.abs(v_hrus[a])*c_hrus[n][a]*zi_hrus[a]*np.sum(gamma[3][a,:]*dy)/hru_area[a]/dx/dy/zi_hrus[a]
        
        fin[a]  =  np.sum(np.abs(u_hrus)*c_hrus[n]*zi_hrus*gamma[0][a,:]*dx)/hru_area[a]/dx/dy/zi_hrus[a] + \
                   np.sum(np.abs(v_hrus)*c_hrus[n]*zi_hrus*gamma[2][a,:]*dy)/hru_area[a]/dx/dy/zi_hrus[a] 
    c_hrus[n+1] = c_hrus[n] + dt * (fin-fout)


cmap = plt.get_cmap('Set3')
colors = [cmap(i) for i in range(0, k, 1)]
new_cmap = ListedColormap(colors)
fig = plt.figure(figsize=(6,6),dpi=100)
fig.set_facecolor('white')
plt.title('HRUs',fontsize=18)
plt.imshow(hrus,origin='lower',cmap=new_cmap)
cb = plt.colorbar(ticks=np.arange(0,k,1),shrink=0.8)
cb.ax.tick_params(labelsize=13)
plt.clim(-0.5, k-0.5)
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
c_maps = np.zeros((nts,hrus.shape[0],hrus.shape[1]))
for n in range(0,nts,1):  
    c_map = np.zeros(hrus.shape)-9999
    for hru in uhrus:
        m = hrus == hru
        c_map[m] = c_hrus[n,hru]
    c_map = np.ma.masked_array(c_map,c_map==-9999) 
    c_maps[n] = c_map
    
for t in range(0,nts,2): 
    fig = plt.figure(figsize=(4,3))
    plt.imshow(np.flipud(c_maps[t]),vmin=0,vmax=1,cmap='coolwarm',alpha=0.9) #vmin=0,vmax=1,
    cb = plt.colorbar(shrink=0.8)
    cb.ax.tick_params(labelsize=12)
    plt.title('Concentration at t = %d s'%(t*dt),fontsize=13)
    plt.xticks([])
    plt.yticks([])
    plt.show()

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.ticker as tkr

plt.rcParams["figure.autolayout"] = True
fig,ax = plt.subplots(figsize=(6,5),facecolor='w')
div = make_axes_locatable(ax)
cax = div.append_axes('right', '5%', '5%')
im = ax.imshow(c_maps[0], origin='lower',vmin=0,vmax=1,cmap='coolwarm',alpha=0.9) 
cb = fig.colorbar(im,cax=cax, format=tkr.FormatStrFormatter('%.1f'))
cb.ax.tick_params(labelsize=15)
tx = ax.set_title("HRU C at t = 0 s",fontsize=18)
ax.set_xticks([])
ax.set_yticks([])
plt.show()

def animate(i):
    cax.cla()
    t = i*2
    im = ax.imshow(c_maps[t], origin='lower',vmin=0,vmax=1,cmap='coolwarm',alpha=0.9) 
    fig.colorbar(im, cax=cax, format=tkr.FormatStrFormatter('%.1f'))
    tx.set_text("HRU C at t = %d s"%(int(t)))
    #ax.set_xticks([])
    #ax.set_yticks([])
ani = animation.FuncAnimation(fig, animate, frames=c_maps.shape[0]//2)
ani